# Скрипт для предобработки (в разработки)

### __[Ссылка на POA](https://docs.google.com/document/d/1roYsQNsmZ2y2lLftn3QCgG293wpQBBlIH48KzK03GBk/edit?usp=sharing)__

### __[Исходный код на GitHub](https://github.com/iyves/ru_col_suggest/tree/master/ru_col_suggest/)__


TODO для предобработки
---
- ? Исключить первую страницу (чтобы удалить адреса емайлы, ссылки, DOIs, имени профессора, названия университетов и тд)
- Переводить все пунктуации на точку (".")
- Разработать unit tests для проверки, что отсутствуют простые ошибки
- Удалить текст из таблиц, рисунок, captions и алгортимов
- Удалить коррумпированный текст
- Удалить нетипично короткие и длинные абзацы
- Разбираться с предложениями, в которых присутствуют иностранные слова ([UNK])
- Дописать документацию
- Удалить неиспользуемый код 
- Параллелизация: разработать код для предобработки несколько статьей одновременно 

In [1]:
import logging
import math
import os
import re
import sys

from abc import ABC, abstractmethod
from bs4 import BeautifulSoup, SoupStrainer
from scripts.preprocessing.kutuzov import rus_preprocessing_udpipe
from string import punctuation
punctuation += 'ʹ…〈〉«»—„“'
from typing import List

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rus_preprocessing_udpipe.download_udpipe_model("./models/")

# Load text from files
encoding = 'utf-8'
def get_text(file_name):
    with open(file_name, "r", encoding=encoding) as f:
        return f.read()


# text = get_text("./ling_cleaned_final/02.txt")
# intext_fragment = re.compile("[[(][^\])]*?[\.\n]") # all characters starting with [ or ( that do not have a ) or ] before reaching a period or new line
# ends_with_number = re.compile("(?:[Ф-яЁё]+?)[0-9]+")
# single_lines = re.compile("(?:\r\r|\n\n|\r\n\r\n|^)(.*)(?=\r\r|\n\n|\r\n\r\n)", re.MULTILINE)



class AbstractPreprocessor(ABC):
    references_start = r"((([1-9][0-9]*(\.[1-9][0-9]*)*)|[ilvVxX]+)(\.?)(\s+))?"
    references_en = r"(Reference|REFERENCE|References|REFERENCES|Bibliography|BIBLIOGRAPHY)"
    references_ru = r"(Л ?и ?т ?е ?р ?а ?т ?у ?р ?а|Л ?И ?Т ?Е ?Р ?А ?Т ?У ?Р ?А|" \
        r"С ?п ?и ?с ?о ?к +л ?и ?т ?е ?р ?а ?т ?у ?р ?ы|С ?П ?И ?С ?О ?К +Л ?И ?Т ?Е ?Р ?А ?Т ?У ?Р ?Ы|" \
        r"И ?с ?т ?о ?ч ?н ?и ?к ?и|И ?С ?Т ?О ?Ч ?Н ?И ?К ?И)"
    references = re.compile(r"{}{}.{{0,80}}$".format(references_start, references_ru), re.MULTILINE | re.DOTALL)
    
    @abstractmethod
    def __init__(self, text, filename):
        self.text = text
        self.filename = filename
    
    @abstractmethod
    def print_text(self):
        return NotImplemented
    
    @abstractmethod
    def remove_references(self):
        return NotImplemented
    
    @abstractmethod
    def remove_header_footer(self):
        return NotImplemented
    
    @abstractmethod
    def remove_footnotes(self):
        return NotImplemented
    
    @abstractmethod
    def tokenize(self):
        return NotImplemented
    
    @abstractmethod
    def preprocess(self):
        return NotImplemented
    
    

class HtmlPreprocessor(AbstractPreprocessor):
    """Preprocesses text in HTML format extracted via the PDFBox tool.
    """
    
    bold_and_italics = re.compile(r"(<b>.*?</b>)|(<i>.*?</i>)", re.MULTILINE | re.DOTALL)
    new_paragraph = re.compile(r"\n+\s+")
    paragraph = re.compile(r"<p>.*?</p>", re.MULTILINE | re.DOTALL)
    
    has_newline = re.compile(r"\n")
    
    footnote = re.compile(r"^[1-9]?[0-9] .+")
    
    # end-of-line (eol)
    eol_hyphenation_in_paragraph = re.compile(r"-\n")
    eol_hyphenation_between_paragraph = re.compile(r"-$")
    
    end_of_sentence = re.compile(r"([!?])|(:\s*$)")
    paragraph_ends_with_period = re.compile(r"\.\s*$")
    
    footnote_reference = re.compile(r"\d+(?:[.,])|(?:[^\s\d])+?\d+(?:\s)", re.MULTILINE)
    
    enumerated = r"\([1-9]?[0-9]\)"
    hyphen_quoted = r"—"
    quoted_example = re.compile(r"(?=((?:\.)\s*({}|{}).+?\.))".format(enumerated, hyphen_quoted), re.MULTILINE)
    enumerated_example = re.compile(r"^({}|\s*{})".format(enumerated, hyphen_quoted), re.MULTILINE)
    
    alpha_cyrillic = "А-яЁё0-9"
    
#     not_alpha_cyrillic_and_punctuation = re.compile("(?=((?:^|[ \n{0}])[{1}]*?[^{1}]+?.*?(?:$|[ \n{0}])))".format(punctuation, alpha_cyrillic), re.MULTILINE)
    not_alpha_cyrillic_and_punctuation = re.compile(r"(?![ \n ! \"#$%&'()*+,-.\/:;<=>?@[\]^_`{|}~ʹ…〈〉«»—„“])[XVIА-яЁё0-9]*?[^XVIА-яЁё0-9 \n ! \"#$%&'()*+,-.\/:;<=>?@[\]^_`{|}~ʹ…〈〉«»—„“]+?.*?(?=$|[ \n ! \"#$%&'()*+,-.\/:;<=>?@[\]^_`{|}~ʹ…〈〉«»—„“])", re.MULTILINE)
    numbers = re.compile(r"(?![ \n ! \"#$%&'()*+,-.\/:;<=>?@[\]^_`{|}~ʹ…〈〉«»—„“])[XVI0-9]+?(?=$|[ \n ! \"#$%&'()*+,-.\/:;<=>?@[\]^_`{|}~ʹ…〈〉«»—„“])", re.MULTILINE)
    
    @classmethod
    def remove_bold_and_italics(cls, text: str) -> str:
        return re.sub(cls.bold_and_italics, "", text)
    
    @classmethod
    def paragraphize(cls, paragraphs: List[str]) -> List[str]:
        paragraphs = [re.sub(cls.new_paragraph, "</p><p>", para) for para in paragraphs]
        paragraphs = re.findall(cls.paragraph, "".join(paragraphs))
        paragraphs = [para[3:-4].strip() for para in paragraphs] # Remove the wrapping <p></p>
        return paragraphs
    
    # From https://stackoverflow.com/questions/14596884/remove-text-between-and-in-python/14598135#14598135
    @classmethod
    def remove_nested_parenteses_brackets(cls, test_str):
        ret = ''
        skip1c = 0
        skip2c = 0
        for i in test_str:
            if i == '[':
                skip1c += 1
            elif i == '(':
                skip2c += 1
            elif i == ']' and skip1c > 0:
                skip1c -= 1
            elif i == ')'and skip2c > 0:
                skip2c -= 1
            elif skip1c == 0 and skip2c == 0:
                ret += i
        return ret
    
    def __init__(self, text, filename):
        self.filename = filename
        self.text = BeautifulSoup(text, 'html.parser', parse_only=SoupStrainer("body"))
        
        self.text = [self.paragraphize([rus_preprocessing_udpipe.unify_sym(str(paragraph))
                                        for paragraph in page.div.find_all('p', recursive=False)])
                     for page in self.text.body.find_all('div', recursive=False)]

    def paragraph_length_percentile(self, k_l: float=0.5, k_u: float=0.5):
        lengths = []
        [[lengths.append(len(para)) for para in page] for page in self.text]
        lengths.sort()
        n = len(lengths)
        index_l = math.ceil(k_l*n)
        index_u = math.ceil(k_u*n)
        return (lengths[index_l], lengths[index_u])
    
    def word_length_percentile(self, k_l: float=0.5, k_u: float=0.5):
        lengths = []
        [[[lengths.append(len(word)) for word in para.split()] for para in page] for page in self.text]
        lengths.sort()
        n = len(lengths)
        index_l = math.ceil(k_l*n)
        index_u = math.ceil(k_u*n)
        return (lengths[index_l], lengths[index_u])
        
    
    def print_text(self):
        logging.info("Text for {}:".format(self.filename))
        for idx, page in enumerate(self.text, 1):
            print("\n", "-"*50, "Start of Page", idx, "-"*50, "\n")
            for paragraph in page:
                print(paragraph, "\n-----")
            print("\n", "-"*50, "End of Page", idx, "-"*50, "\n")
    
    def remove_references(self):
        for i in reversed(range(len(self.text))):
            for j in reversed(range(len(self.text[i]))):
                if re.search(self.references, self.text[i][j]):
                    self.text[i] = self.text[i][:j]
                    self.text = self.text[:i+1]
                    return True
        return False
    
    def remove_styled_text(self):
        for page in self.text:
            for j in range(len(page)):
                page[j] = self.remove_bold_and_italics(page[j])
    
    def remove_empty_paragraphs(self):
        for i in reversed(range(len(self.text))):
            for j in reversed(range(len(self.text[i]))):
                if self.text[i][j].strip() == "":
                    self.text[i].pop(j)
            if len(self.text[i]) < 1:
                self.text.pop(i)
   
    def remove_header_footer(self):
        # WARNING: Removes the first(and possibly second) line if it has no newlines -- this will be incorrect in some cases!
        for i in range(len(self.text)):
            if not re.search(self.has_newline, self.text[i][0]):
                if len(self.text[i]) > 1:
                    if not re.search(self.has_newline, self.text[i][1]):
                        self.text[i] = self.text[i][2:] # Header and footer
                    else:
                        self.text[i] = self.text[i][1:] # Either just header or footer
                else:
                    self.text.pop(i)
    
    def remove_footnotes(self):
        # Search each page for a footnote match, and then remove any lines lower than that
        for i in range(len(self.text)):
            for j in range(len(self.text[i])):
                if re.search(self.footnote, self.text[i][j]):
                    self.text[i] = self.text[i][:j]
                    break
    
    def remove_eol_hyphenation(self):
        # Hyphenation within paragraphs
        for page in self.text:
            for j in range(len(page)):
                page[j] = re.sub(self.eol_hyphenation_in_paragraph, "", page[j])
                page[j] = re.sub("\n", " ", page[j])
        
        # Hyphenation between paragraphs
        for page in self.text:
            for j in reversed(range(1, len(page))):
                if page[j-1][-1] == "-":
                    page[j-1] = page[j-1][:-1] + page[j]
                    page.pop(j)
        
        # Hyphenation between pages:
        for i in reversed(range(1, len(self.text))):
            if self.text[i-1][-1][-1] == "-":
                self.text[i-1][-1] = self.text[i-1][-1][:-1] + self.text[i][0]
                
                if len(self.text[i]) > 1:
                    self.text[i] = self.text[i][1:]
                else:
                    self.text.pop(i)
    
    def end_of_sentence_to_period(self):
        for i in range(len(self.text)):
            self.text[i] = [re.sub(self.end_of_sentence, ".", para) for para in self.text[i]]

    def join_broken_sentences(self):
        # Sentence broken between paragraphs
        for page in self.text:
            for j in reversed(range(1, len(page))):
                if not re.search(self.paragraph_ends_with_period, page[j-1]):
                    page[j-1] = page[j-1] + page[j]
                    page.pop(j)
        
        # Sentence broken between pages:
        for i in reversed(range(1, len(self.text))):
            if not re.search(self.paragraph_ends_with_period, self.text[i-1][-1]):
                self.text[i-1][-1] = self.text[i-1][-1] + self.text[i][0]
                
                if len(self.text[i]) > 1:
                    self.text[i] = self.text[i][1:]
                else:
                    self.text.pop(i)
    
    def remove_examples(self):
        # Lines that start with enumerated 
        for i in range(len(self.text)):
            for j in reversed(range(len(self.text[i]))):
                if re.search(self.enumerated_example, self.text[i][j]):
                    self.text[i].pop(j)
    
    def remove_intext_references(self):
        # Bracketed and parenthesized text
        for i in range(len(self.text)):
            self.text[i] = [self.remove_nested_parenteses_brackets(para)
                            for para in self.text[i]]
        
        # References to footnotes
        for i in range(len(self.text)):
            self.text[i] = [re.sub(self.footnote_reference, "", para) for para in self.text[i]]
    
    def replace_oov(self):
        for i in range(len(self.text)):
            self.text[i] = [re.sub(self.not_alpha_cyrillic_and_punctuation, "[UNK]", para) for para in self.text[i]]
    
    def replace_numbers(self):
        for i in range(len(self.text)):
            self.text[i] = [re.sub(self.numbers, "[NUM]", para) for para in self.text[i]]
        
    
    def remove_short_and_long_paragraphs(self):
        paragraph_bounds = preprocessor.paragraph_length_percentile(.1,.9)
        word_bounds = preprocessor.word_length_percentile(.25,.75)
        tokens = self.tokenize()
        print("para bounds:", paragraph_bounds, "\nword bounds:", word_bounds)
        for page in tokens:
            for para in page:
                print (para)
                avg_word = [len(word) for word in para]
                avg_word = sum(avg_word) / len(avg_word)
                print("cur word:", avg_word)
                if avg_word < word_bounds[0] or avg_word > word_bounds[1]:
                    print("word: REMOVE")
                print("\n\n")
            print("\n\n\n\n\n")
#         for i in range(len(self.text)):
#             avg_para = [len(para) for para in self.text[i]]
#             avg_para = sum(avg_para) / len(avg_para)
            
#             avg_word = [[len(word) for word in para.split()] for para in self.text[i]]
#             avg_word = [sum(sub_word) / len(sub_word) for sub_word in avg_word]
#             avg_word = sum(avg_word) / len(avg_word)
#             print(self.text[i], "\n")
#             print("cur para:", avg_para, "; cur word:", avg_word)
#             if avg_para < paragraph_bounds[0] or avg_para > paragraph_bounds[1]:
#                 print("para: REMOVE")
#             if avg_word < word_bounds[0] or avg_word > word_bounds[1]:
#                 print("word: REMOVE")
#             print("\n\n")
#         for page in self.text:
#             for para in page:
#                 print(para)
#                 for sentence in para:
#                     print (sentence)
#                 avg_word = [len(word) for word in page[j].split()]
#                 avg_word = sum(avg_word) / len(avg_word)
#                 print(self.text[j], "\n")
#                 print("cur word:", avg_word)
#                 if avg_word < word_bounds[0] or avg_word > word_bounds[1]:
#                     print("word: REMOVE")
#                 print("\n\n")
        
    def tokenize(self, keep_pos: bool = False, 
                 keep_punct: bool = True):
        """Lemmatize and tokenize a text paragraph by paragraph with udpipe.
        
        Args:
            text:
                The text to tokenize. Paragraphs are delimited by a single newline character
            keep_pos:
                Optional; Append the part of speech tag to the end of each token or
                not. The default behavior is to drop the POS tag.
            keep_punct:
                Optional; Keep punctuation marks as separate tokens. The default
                behavior is to keep punctuation.
        
        Returns:
            A list of each paragraph, where each paragraph is a list of lemmatized tokens. 
        """
        tokens: List[List[List[str]]] = [[rus_preprocessing_udpipe.process(
            rus_preprocessing_udpipe.process_pipeline, text=paragraph,
            keep_pos=keep_pos, keep_punct=keep_punct) + ['\n']
                  for paragraph in page] for page in self.text]
 
        return tokens
    
    def preprocess(self):
        if not self.remove_references():
            logging.error("Could not find the references section for file: {}".format(self.filename))
        self.remove_footnotes()
        self.remove_styled_text()
        self.remove_header_footer()
        self.remove_empty_paragraphs()
        self.remove_eol_hyphenation()
        self.end_of_sentence_to_period()
        self.join_broken_sentences()
        self.remove_examples()
        self.remove_intext_references()
        self.replace_oov()
        self.replace_numbers()
#         self.remove_short_and_long_paragraphs()



# Set up the article to preprocess
filename = "49.html"
text = get_text("../data/extracted/html/linguistics/" + filename)

# Set up the preprocesser and preprocess the article
preprocessor = HtmlPreprocessor(text, filename)
preprocessor.preprocess()
preprocessor.print_text()

# print(preprocessor.paragraph_length_percentile(.1,.9))
# print(preprocessor.word_length_percentile(.25,.75))

# # Tokenization via Kutuzov's scripts
# tokens = preprocessor.tokenize(keep_pos = False, keep_punct = True)
# for page in tokens:
#     for para in page:
#         print(' '.join(para))
#         print("\n")
#     print("\n\n", '-'*50, "\n\n")


UDPipe model not found. Downloading...

Loading the model...
2020-09-15 11:09:43,013 : INFO : Text for 49.html:



 -------------------------------------------------- Start of Page 1 -------------------------------------------------- 

[UNK] Санкт-Петербургский государственный университет, 2017УДК 1Санкт-Петербургский государственный университет,   Российская Федерация,  Санкт-Петербург, Университетская наб., [NUM] -- [NUM]; Институт филологических исследований РАН,   Российская Федерация,  Санкт-Петербург, Тучков пер., [NUM]   [UNK]@[UNK].[UNK] КРАТКИХ ДЕЙСТВИТЕЛЬНЫХ ПРИЧАСТИЙ   В РОЛИ ВТОРОСТЕПЕННОГО СКАЗУЕМОГО   В ПЕРЕВОДНОМ ЖИТИЙНОМ ТЕКСТЕ   Статья посвящена изучению закономерностей употребления причастий в  функции второстепенного сказуемого в  переводном памятнике, относящемся к  эпохе второго южнославянского влияния. Сравнение выявленных закономерностей с восточнославянскими средневековыми тенденциями, характерными для повествовательных жанров, позволило определить  общие направления их реализации. Кроме того, при анализе отклонений славянского перевода  от греческого оригинала удалось обна

# Liza's preprocessing script

In [20]:
from scripts.preprocessing.liza import CAT_preprocessing

# Set up the article to preprocess
filename = "49.txt"
text = get_text("../data/extracted/txt/linguistics/" + filename)

print(CAT_preprocessing.clean_text(text))

пространение на Руси (в настоящий момент известно 29 русских списков жития, самые ранние из которых датируются XV в.), появившись в русской книжности в эпоху так называемого второго южнославянского влияния [Сперанский, с. 101; Кенанов, с. Указанное обстоятельство позволяет рассматривать переводное "Житие Николая Мирликийского" (далее — "ЖНМ") с точки зрения проявления в нем языковых признаков, свойственных русскому литературному языку данной эпохи. Обращение к переводному памятнику дает возможность оценить границы влияния греческого языка на церковнославянский и

таким образом определить происхождение языковых явлений в славянском переводе.

Актуальность подобного исследования связана с

проблемами, сформулиро- ванными Д. Вортом: выявлением круга собственно лингвистических компонентов, характерных для произведений, относящихся к

эпохе второго южнославянского влияния, и характеристикой их происхождения : эллинизмы (например, графемы греческого алфавита), архаизмы (например, возвращени